In [ ]:
from traitlets import Unicode, Bool, validate, TraitError, observe
from ipywidgets import DOMWidget, register
import time
import threading

In [ ]:
def secret_python_function(arg0, arg1):
    print(f"SecretPythonPrint: {arg0}, {arg1}")
    return f"SecretPythonReturn: {arg0}, {arg1}"

In [ ]:
%%javascript

require.undef('email_widget');

define('email_widget', ["@jupyter-widgets/base"], function(widgets) {
    var WebVisualizerView = widgets.DOMWidgetView.extend({
        sleep: function(time_ms) {
            return new Promise((resolve) => setTimeout(resolve, time_ms));
        },
        
        jspy_send: function(message) {
            this.model.set("jspy_channel", message);
            this.touch();
        },
        
        // args must be all strings.
        // kwargs must be all string:string.
        // TODO: sanity check?
        callPython: async function(func, args=[], kwargs={}) {
            var message = {
                function: func,
                args: args,
                kwargs: kwargs
            };
            this.jspy_send(JSON.stringify(message));
        },
        
        render: function() {
            this.email_input = document.createElement('p');
            this.email_input.innerText = this.model.get('jspy_channel');
            this.el.appendChild(this.email_input);
            
            // Listen for py->js message.
            this.model.on('change:pyjs_channel', this.on_pyjs_message, this);
            
            // Send several js->py message for testing.
            this.sleep(1000).then(() => {
                this.model.set('jspy_channel', "Hello from JS: A");
                this.touch();
            }).then(() => this.sleep(1000)).then(() => {
                this.model.set('jspy_channel', "Hello from JS: B");
                this.touch();
            }).then(() => this.sleep(1000)).then(() => {
                this.model.set('jspy_channel', "Hello from JS: C");
                this.touch();
            });
        },
        
        on_pyjs_message: function() {
            this.email_input.innerText = this.model.get('pyjs_channel');
        },
        
    });
    
    return {
        WebVisualizerView: WebVisualizerView
    }
});

In [ ]:
def py2js_send_five_messages(widget):
    def func():
        for i in range(3):
            widget.pyjs_channel = f"Hello from Python: {i+1}"
            time.sleep(1)
    return func

@register
class WebVisualizer(DOMWidget):
    _view_name = Unicode('WebVisualizerView').tag(sync=True)
    _view_module = Unicode('email_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    
    # Attributes
    pyjs_channel = Unicode("Empty pyjs_channel.", help="Python->JS message channel.").tag(sync=True)
    jspy_channel = Unicode("Empty jspy_channel.", help="JS->Python message channel.").tag(sync=True)

    @observe('jspy_channel')
    def on_jspy_message(self, change):
        print(f"jspy message: {change['new']}")

In [ ]:
visualizer = WebVisualizer()
visualizer

In [ ]:
thread = threading.Thread(target=py2js_send_five_messages(visualizer))
thread.start()